# 在 Amazon Comprehend 中利用预训练的 API 

## 目录
- [介绍](#intro)
- [安装](#setup)
- [识别命名实体](#identifying-named-entities)
- [检测关键短语](#detecting-key-phrases)
- [检测主要语言](#identifying-the-ominant-language)
- [情感分析](#determining-emotional-sentiment)
- [语法检测](#determiningsyntax)
- [检测个人身份信息 (PII)](#detecting-pii)
- [总结](#conclusion)
- [清理](#clean-up)

## <a name='intro' ></a>介绍

此 Notebook 示例提供了使用 [Amazon Comprehend](https://aws.amazon.com/comprehend/) 的预训练 API 来发现非结构化数据中的信息的步骤说明。 Amazon Comprehend 使用预先训练的模型来检查和分析一个文档或一组文档以收集见解。该模型在大量文本上不断训练，因此您无需提供训练数据。

我们将探索 6 个预训练的 API：识别命名实体、提取关键短语、识别主要语言、确定情绪、确定语法、检测检测个人身份信息 (PII)。

此示例使用 AWS 的资源，您在运行时可能会产生费用。 

## <a name="tips" ></a>提示

如果你是 Python Notebooks 的新手：`SHIFT` + `ENTER` 将执行一个代码单元并转到下一个。 

## <a name='setup' ></a>安装
我们导入相关依赖包以与 Amazon Comprehend 交互。 [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) 是 AWS Python 开发工具包。 

In [ ]:
import boto3

我们指定 SageMaker 执行角色，这是此笔记本中使用的角色，并且使用笔记本所在的AWS区域执行Comprehend任务。 

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
region = boto3.Session().region_name

导入我们将使用的其他包。 

In [ ]:
import json
import pandas as pd
import numpy as np
import pprint

## 启动 Amazon Comprehend 客户端 

In [ ]:
comprehend = boto3.client(service_name='comprehend', region_name=region)

## 数据

在本实验中，我们将使用 [Amazon Comprehend 控制台] (https://console.aws.amazon.com/comprehend) 中使用的相同示例输入文本（英文版和中文版）。如果您想尝试不同的文本，只需修改下面的单元格。 

In [ ]:
sample_text = '''
Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.
'''

In [ ]:
cn_sample_text = '''
你好张伟。 您的汇丰银行信用卡账户 1111-0000-1111-0000 的最低付款额为 24.53 美元，到期日期为 7 月 31 日。 根据您的自动转账设置，我们将在到期日从您的银行账户 XXXXXX1111 中提取您的付款，路由号码为 XXXXX0000。
您的最新声明已邮寄至上海市长宁区中山公园。
收到付款后，您将在 206-555-0100 收到确认短信。
如果您对帐单有任何疑问，请致电 206-555-0199 或发送电子邮件至 support@anycompany.com 联系客户服务。
'''

## <a name='identifying-named-entities' ></a>识别命名实体

命名实体是可以用专有名称表示的真实世界对象（人、地点、位置、组织等）。

Amazon Comprehend 可以从文档或文本中提取命名实体。 例如，这对于索引、文档标签或搜索很有用。 有关更多信息，请参阅 [检测实体](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectEntities.html))。

用于提取这些实体的 API 是 [DetectEntities API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectEntities)。

对于检测到的每个实体，Amazon Comprehend 返回类型，例如“Person”或“Date”，以及在此检测中的置信度分数。 在您的实现中，您可以使用此置信度分数来设置阈值。 

In [ ]:
print('Calling DetectEntities')
detected_entities = comprehend.detect_entities(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_entities, sort_keys=True, indent=4))
print('End of DetectEntities\n')

In [ ]:
cn_detected_entities = comprehend.detect_entities(Text=cn_sample_text, LanguageCode='zh')

In [ ]:
cn_detected_entities

返回结果包括分数、类型和偏移量。

现在我们让它更具可读性：

In [ ]:
detectec_entities_df = pd.DataFrame([ [entity['Text'], entity['Type'], entity['Score']] for entity in detected_entities['Entities']],
                columns=['Text', 'Type', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detectec_entities_df)

In [ ]:
cn_detectec_entities_df = pd.DataFrame([ [entity['Text'], entity['Type'], entity['Score']] for entity in cn_detected_entities['Entities']],
                columns=['Text', 'Type', 'Score'])

print('This was the text analyzed:')
print(cn_sample_text)
print()
display (cn_detectec_entities_df)

## <a name='detecting-key-phrases' ></a>识别关键短语

Amazon Comprehend 可以提取出现在文档中的关键名词短语。 例如，关于篮球比赛的文档可能会返回球队名称、场地名称和最终得分。这可以用于索引或摘要。有关更多信息，请参阅 [检测关键短语](https://docs.aws.amazon.com/comprehend/latest/dg/get-started-api-key-phrases.html)。

用于提取这些关键短语的 API 是 [DetectKeyPhrases API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectKeyPhrases)。

Amazon Comprehend 返回关键短语以及表明模型在此检测中的置信度分数。在您的实现中，您可以使用此置信度分数来设置阈值。 

In [ ]:
print('Calling DetectKeyPhrases')
detected_key_phrases = comprehend.detect_key_phrases(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_key_phrases, sort_keys=True, indent=4))
print('End of DetectKeyPhrases\n')

In [ ]:
print('Calling DetectKeyPhrases')
cn_detected_key_phrases = comprehend.detect_key_phrases(Text=cn_sample_text, LanguageCode='zh')
pprint.pprint(cn_detected_key_phrases)
print('End of DetectKeyPhrases\n')

返回结果包括分数、类型和偏移量。

现在我们让它更具可读性：

In [ ]:
detected_key_phrases_df = pd.DataFrame([ [entity['Text'], entity['Score']] for entity in detected_key_phrases['KeyPhrases']],
                columns=['Text', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_key_phrases_df)

In [ ]:
cn_detected_key_phrases_df = pd.DataFrame([ [entity['Text'], entity['Score']] for entity in cn_detected_key_phrases['KeyPhrases']],
                columns=['Text', 'Score'])

print('This was the text analyzed:')
print(cn_sample_text)
print()
display (cn_detected_key_phrases_df)

## <a name='identifyinf-the-dominant-language'></a> 识别主要语言

Amazon Comprehend 可以识别文档中的主要语言。 Amazon Comprehend 目前可以识别多种语言。 这可以作为进一步处理之前的第一步，例如当电话记录可以是不同的语言时。 有关更多信息，包括可以识别哪些语言，请参阅[检测主要语言](https://docs.aws.amazon.com/comprehend/latest/dg/how-languages.html)。

用于识别主导语言的 API 是 [DetectDominantLanguage API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectDominantLanguage)。

Amazon Comprehend 返回主要语言以及表明模型在此检测中的置信度分数。在您的实现中，您可以使用此置信度分数来设置阈值。 如果检测到一种以上的语言，它将返回每种检测到的语言及其相应的置信度分数。 

In [ ]:
print('Calling DetectDominantLanguage')
detected_language = comprehend.detect_dominant_language(Text=sample_text)
print(json.dumps(detected_language, sort_keys=True, indent=4))
print('End of DetectDominantLanguage\n')

In [ ]:
print('Calling DetectDominantLanguage')
cn_detected_language = comprehend.detect_dominant_language(Text=cn_sample_text)
print(json.dumps(cn_detected_language, sort_keys=True, indent=4))
print('End of DetectDominantLanguage\n')

返回结果包括分数和检测到的语言代码。

现在我们让它更具可读性： 

In [ ]:
detected_language_df = pd.DataFrame([ [code['LanguageCode'], code['Score']] for code in detected_language['Languages']],
                columns=['Language Code', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_language_df)

In [ ]:
cn_detected_language_df = pd.DataFrame([ [code['LanguageCode'], code['Score']] for code in cn_detected_language['Languages']],
                columns=['Language Code', 'Score'])

print('This was the text analyzed:')
print(cn_sample_text)
print()
display (cn_detected_language_df)

## <a name='determining-emotional-sentiment' ></a>情感分析

Amazon Comprehend 可以确定文档的情感。情感可以是积极的、中性的、消极的或混合的。 有关详细信息，请参阅情感分析。 例如，这对于分析来自呼叫中心的评论内容非常有用。 有关更多信息，请参阅[情感检测](https://docs.aws.amazon.com/comprehend/latest/dg/get-started-api-sentiment.html)。

用于提取情感的 API 是 [DetectSentiment API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment)。

Amazon Comprehend 返回不同的情感结果以及每种情感的相关置信度分数。具有最高置信度分数的情感可以被视为文本中的主要情感。 

In [ ]:
print('Calling DetectSentiment')
detected_sentiment = comprehend.detect_sentiment(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_sentiment, sort_keys=True, indent=4))
print('End of DetectSentiment\n')

In [ ]:
print('Calling DetectSentiment')
cn_detected_sentiment = comprehend.detect_sentiment(Text=cn_sample_text, LanguageCode='zh')
print(json.dumps(cn_detected_sentiment, sort_keys=True, indent=4))
print('End of DetectSentiment\n')

返回结果包括主要情感和每个检测到的情感的分数。

现在我们让它更具可读性： 

In [ ]:
predominant_sentiment = detected_sentiment['Sentiment']
detected_sentiments_df = pd.DataFrame([ [sentiment, detected_sentiment['SentimentScore'][sentiment]] for sentiment in detected_sentiment['SentimentScore']],
                columns=['Language Code', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
print('The predominant sentiment is {}.'.format(predominant_sentiment))
print()
display (detected_sentiments_df)

In [ ]:
cn_predominant_sentiment = cn_detected_sentiment['Sentiment']
cn_detected_sentiments_df = pd.DataFrame([ [sentiment, cn_detected_sentiment['SentimentScore'][sentiment]] for sentiment in detected_sentiment['SentimentScore']],
                columns=['Language Code', 'Score'])

print('This was the text analyzed:')
print(cn_sample_text)
print()
print('The predominant sentiment is {}.'.format(cn_predominant_sentiment))
print()
display (cn_detected_sentiments_df)

## <a name="determining-syntax" ></a> 检测语法

Amazon Comprehend 会解析文档中的每个单词并确定该单词的语法和词性。目前该功能还不支持中文。例如，在句子“It is raining today in Seattle”中，“it”被识别为代词，“raining”被识别为动词，“Seattle”被识别为专有名词。 有关更多信息，请参阅 [分析语法](https://docs.aws.amazon.com/comprehend/latest/dg/how-syntax.html)。

用于提取语法信息的 API 是 [DetectSyntax API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSyntax)。

Amazon Comprehend 返回不同的词类以及每个词类的相关置信度分数。 

In [ ]:
print('Calling DetectSyntax')
detected_syntax = comprehend.detect_syntax(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_syntax, sort_keys=True, indent=4))
print('End of DetectSyntax\n')

Amazon Comprehend 返回文本、不同的词性、每个词的置信度分数和在文中的位置。

现在我们让它更具可读性： 

In [ ]:
detected_syntax_df = pd.DataFrame([ [part['Text'], part['PartOfSpeech']['Tag'], part['PartOfSpeech']['Score']] for part in detected_syntax['SyntaxTokens']],
                columns=['Text', 'Part Of Speech', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
print('First twenty tokens:')
display (detected_syntax_df.head(20))

## <a name='detecting-pii' ></a> 检测个人身份信息 (PII)

Amazon Comprehend 可以分析文档检测个人信息数据，例如地址、银行帐号或电话号码。这对于信息提取和索引以及遵守有关数据保护的法律要求很有用。有关更多信息，请参阅[检测个人身份信息 (PII)](https://docs.aws.amazon.com/comprehend/latest/dg/how-pii.html)。

Amazon Comprehend 可以帮助您识别文档中单个 PII 的位置，或帮助您标记包含 PII 的文档。 

### 识别文本中 PII 的位置

Amazon Comprehend 可以帮助您识别文档中各个 PII 的位置。

用于识别个人 PII 位置的 API 是 [DetectPiiEntities API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectPiiEntities.html)。

Amazon Comprehend 返回不同的 PII 和每个 PII 的相关置信度分数。 

In [ ]:
print('Calling DetectPiiEntities')
detected_pii_entities = comprehend.detect_pii_entities(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_pii_entities, sort_keys=True, indent=4))
print('End of DetectPiiEntities\n')

Amazon Comprehend 返回 PII 实体、每个实体的置信度分数和偏移量。

现在我们让它更具可读性： 

In [ ]:
detected_pii_entities_df = pd.DataFrame([ [entity['Type'], entity['Score']] for entity in detected_pii_entities['Entities']],
                columns=['Type', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_pii_entities_df)

### 使用 PII 标记文本

Amazon Comprehend 可以帮助您标记包含 PII 的文档。 在个人身份信息 (PII) 分析模式中选择“标签”。

用于提取文档中 PII 实体的 API。 我们使用了 [ContainsPiiEntities API](https://docs.aws.amazon.com/comprehend/latest/dg/API_ContainsPiiEntities.html)。

Amazon Comprehend 返回不同的 PII 标签和每个标签的相关置信度分数。这些标签表明文档中存在这些类型的 PII。 

In [ ]:
print('Calling ContainsPiiEntities')
detected_pii_labels = comprehend.contains_pii_entities(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_pii_labels, sort_keys=True, indent=4))
print('End of ContainsPiiEntities\n')

Amazon Comprehend 返回 PII 实体名称和完整分数。

现在我们让它更具可读性： 

In [ ]:
detected_pii_labels_df = pd.DataFrame([ [entity['Name'], entity['Score']] for entity in detected_pii_labels['Labels']],
                columns=['Name', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_pii_labels_df)

## <a name="conclusion"></a>总结

您现在已经学习了如何使用 Python SDK 使用预训练的 API。

有关如何从 AWS 管理控制台使用这些 API 的示例，请按照Workshop网站中“使用 AWS 管理控制台”中的步骤进行操作。

## <a name="clean-up"></a>清理

使用完此笔记本后，请务必在 [Amazon SageMaker 控制台](https://console.aws.amazon.com/sagemaker/) 中停止并删除此 Amazon SageMaker 笔记本实例，以避免产生额外费用。 